# Xabi López, project 21 SQL

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///../../lab-sql-select/publications.db

'Connected: @../../lab-sql-select/publications.db'

## Challenge 1 - Most Profiting Authors

Step 1: Calculate the royalties of each sales for each author

In [29]:
%%sql
select ta.title_id as Title_ID, aut.au_id as Author_ID, round(tit.price * sal.qty * tit.royalty / 100 * ta.royaltyper / 100) as Royalty
from authors aut inner join titleauthor ta on ta.au_id = aut.au_id
                 inner join sales sal on sal.title_id = ta.title_id
                 inner join titles tit on tit.title_id = ta.title_id
limit 5

 * sqlite:///../../lab-sql-select/publications.db
Done.


Title_ID,Author_ID,Royalty
PS3333,172-32-1176,30.0
BU1032,213-46-8915,4.0
BU1032,213-46-8915,8.0
BU2075,213-46-8915,25.0
PC1035,238-95-7766,110.0


Step 2: Aggregate the total royalties for each title for each author

In [30]:
%%sql
select ta.title_id as Title_ID, aut.au_id as Author_ID, round(sum(tit.price * sal.qty * tit.royalty / 100 * ta.royaltyper / 100)) as Royalty
from authors aut inner join titleauthor ta on ta.au_id = aut.au_id
                 inner join sales sal on sal.title_id = ta.title_id
                 inner join titles tit on tit.title_id = ta.title_id
group by ta.title_id, aut.au_id
limit 5

 * sqlite:///../../lab-sql-select/publications.db
Done.


Title_ID,Author_ID,Royalty
PS3333,172-32-1176,30.0
BU1032,213-46-8915,12.0
BU2075,213-46-8915,25.0
PC1035,238-95-7766,110.0
BU1111,267-41-2394,12.0


Step 3: Calculate the total profits of each author

In [31]:
%%sql
select ta.title_id as Title_ID, aut.au_id as Author_ID, round(sum(tit.price * sal.qty * tit.royalty / 100 * ta.royaltyper / 100)) as Royalty, tit.advance as Advance
from authors aut inner join titleauthor ta on ta.au_id = aut.au_id
                 inner join sales sal on sal.title_id = ta.title_id
                 inner join titles tit on tit.title_id = ta.title_id
group by ta.title_id, aut.au_id
limit 5

 * sqlite:///../../lab-sql-select/publications.db
Done.


Title_ID,Author_ID,Royalty,Advance
PS3333,172-32-1176,30.0,2000
BU1032,213-46-8915,12.0,5000
BU2075,213-46-8915,25.0,10125
PC1035,238-95-7766,110.0,7000
BU1111,267-41-2394,12.0,5000


In [27]:
%%sql
select Author_ID, Royalty + Advance as Profit 

from (select ta.title_id as Title_ID, aut.au_id as Author_ID, round(sum(tit.price * sal.qty * tit.royalty / 100 * ta.royaltyper / 100)) as Royalty, tit.advance as Advance
from authors aut inner join titleauthor ta on ta.au_id = aut.au_id
                 inner join sales sal on sal.title_id = ta.title_id
                 inner join titles tit on tit.title_id = ta.title_id
group by ta.title_id, aut.au_id)

order by Profit desc
limit 3

 * sqlite:///../../lab-sql-select/publications.db
Done.


Author_ID,Profit
722-51-5454,15022.0
899-46-2035,15007.0
213-46-8915,10150.0


## Challenge 2

In [34]:
import pandas as pd

In [32]:
db_url = 'sqlite:///../../lab-sql-select/publications.db'

In [37]:
query = '''
select Author_ID, Royalty + Advance as Profit 

from (select ta.title_id as Title_ID, aut.au_id as Author_ID, round(sum(tit.price * sal.qty * tit.royalty / 100 * ta.royaltyper / 100)) as Royalty, tit.advance as Advance
from authors aut inner join titleauthor ta on ta.au_id = aut.au_id
                 inner join sales sal on sal.title_id = ta.title_id
                 inner join titles tit on tit.title_id = ta.title_id
group by ta.title_id, aut.au_id)

order by Profit desc
'''

In [39]:
df = pd.read_sql(query, db_url)
df.head(3)

,Author_ID,Profit
0,722-51-5454,15022.0
1,899-46-2035,15007.0
2,213-46-8915,10150.0


In [40]:
df.to_sql('most_profiting_authors',db_url,if_exists='replace',index=False)

In [41]:
%%sql
select *
from most_profiting_authors

 * sqlite:///../../lab-sql-select/publications.db
Done.


Author_ID,Profit
722-51-5454,15022.0
899-46-2035,15007.0
213-46-8915,10150.0
427-17-2319,8050.0
846-92-7186,8050.0
672-71-3249,8012.0
267-41-2394,8009.0
472-27-2349,8009.0
238-95-7766,7110.0
807-91-6654,7084.0
